In [1]:
import pandas as pd
import h5py
import os
import numpy as np
import pickle

In [2]:
def get_landmarks(task, session, participant, subset):
    data_path = f'/nas/project_data/B1_Behavior/hakim/UDIVA/annotation/{subset}'
    annotation_path = os.path.join(data_path, task, session, participant, 'annotations_raw.hdf5')
    
    landmarks = []
    with h5py.File(annotation_path, "r") as f:
        for frame in f.keys():
            try:
                landmark = f[frame]['face']['landmarks'][()]
                landmarks.append(landmark)
            except KeyError:
                landmarks.append(None)
        f.close()
    return landmarks

In [3]:
def get_mid_landmarks(landmarks):
    mid_landmarks = []
    for landmark in landmarks:
        mid_landmark = [0, 0]
        if landmark is not None:
            mid_landmark[0] = landmark[0,:].mean()
            mid_landmark[1] = landmark[1,:].mean()
        mid_landmarks.append(np.array(mid_landmark))
    return mid_landmarks

In [4]:
def get_face_moves(mid_landmarks):
    face_moves = []
#     the first move is zero
    face_moves.append(np.array([0,0]))
    for i in range(len(mid_landmarks)-1):
        move = mid_landmarks[i+1] - mid_landmarks[i]
        face_moves.append(move)
    return face_moves

In [5]:
def combine_pose_face_moves(out_3ddfa, face_moves):
    pose_face_moves = []
    for i in range (len(face_moves)):
        com = []
        if out_3ddfa[i] is not None:
            com = list(out_3ddfa[i]['pose']) + list(face_moves[i])
        else:
            com = [0,0,0] + list(face_moves[i])
        pose_face_moves.append(com)
    return np.array(pose_face_moves)

In [11]:
subset = 'test'
# tasks = ['animals', 'ghost', 'lego', 'talk']
tasks = ['lego']

annotation_dir = f'/nas/project_data/B1_Behavior/hakim/UDIVA/annotation/{subset}'
output_3ddfa_dir = f'/nas/project_data/B1_Behavior/hakim/UDIVA/3DDFA/{subset}'
output_dir = f'/nas/project_data/B1_Behavior/hakim/UDIVA/pose_face_moves/{subset}'

for task in tasks:
    anno_path = os.path.join(annotation_dir, task)
    
    for session in os.listdir(anno_path):
        session_path = os.path.join(anno_path, session)
    
        for par in os.listdir(session_path):
            file_name = task + '_' + session + '_' + par[:3]
            out_name = file_name + '.npy'
            out_path = os.path.join(output_dir, out_name)
            out_3ddfa_name = file_name + '.pickle'
            out_3ddfa_path = os.path.join(output_3ddfa_dir, out_3ddfa_name)
            
            if (task == 'talk' and session == '188189' and par == 'FC1_T'):
                continue
                
            if (os.path.exists(out_path)):
                continue
            
            with open(out_3ddfa_path, 'rb') as f:
                out_3ddfa = pickle.load(f, encoding='latin1')
            
            landmarks = get_landmarks(task, session, par, subset)
            mid_landmarks = get_mid_landmarks(landmarks)
            face_moves = get_face_moves(mid_landmarks)
            
            assert len(out_3ddfa) == len(face_moves)
            
            pose_face_moves = combine_pose_face_moves(out_3ddfa, face_moves)
                        
#             with open(out_path, 'wb') as f:
#                 np.save(f, pose_face_moves)

(6944, 5)
(6944, 5)


# Check annotation file

In [51]:
p = '/nas/project_data/B1_Behavior/hakim/UDIVA/annotation/val/ghost/001080/FC1_G/annotations_raw.hdf5'

with h5py.File(p, "r") as f:
    for frame in f.keys():
        try:
            hand = f[frame]['hands']['left']['landmarks'][()]
            body = f[frame]['body']['landmarks'][()]
            face = f[frame]['face']['landmarks'][()]
        except KeyError:
            continue
    f.close()